### Scheduling

_Consider the following set of processes:_

| Process | Duration | Arrival Time |
|---------|----------|--------------|
| A       | 20       | 0            |
| B       | 15       | 0            |
| C       | 25       | 5            |
| D       | 5        | 10           |

* Q11: _Draw the schedule when a First In First Out (FIFO) scheduling algorithm is used._

```
  A                   B              C                        D
|-------------------|--------------|------------------------|----|
0                   20             35                       60   65
```

<p style="height:8em;"></p>

* Q12: _Compute the turnaround and wait time for each process based on the above schedule._

| Process | Turnaround | Wait |
|---------|------------|------|
| A       | 20         | 0    |
| B       | 35         | 20   |
| C       | 55         | 30   |
| D       | 55         | 50   |

| Process | Turnaround | Wait |
|---------|------------|------|
| A       |            |      |
| B       |            |      |
| C       |            |      |
| D       |            |      |

* Q13: _Draw the schedule when a Shortest Job First (SJF) scheduling algorithm is used._

```
  B              D    A                   C
|--------------|----|-------------------|------------------------|
0              15   20                  40                       65
```

<p style="height:8em;"></p>

* Q14: _Compute the turnaround and wait time for each process based on the above schedule._

| Process | Turnaround | Wait |
|---------|------------|------|
| A       | 40         | 20   |
| B       | 15         | 0    |
| C       | 60         | 35   |
| D       | 10         | 5    |

| Process | Turnaround | Wait |
|---------|------------|------|
| A       |            |      |
| B       |            |      |
| C       |            |      |
| D       |            |      |

* Q15: _Draw the schedule when a Shortest Time to Completion First (STCF) scheduling algorithm is used._

```
  B              D    A                   C
|--------------|----|-------------------|------------------------|
0              15   20                  40                       65
```

<p style="height:8em;"></p>

* Q16: _Compute the turnaround and wait time for each process based on the above schedule._

| Process | Turnaround | Wait |
|---------|------------|------|
| A       | 40         | 20   |
| B       | 15         | 0    |
| C       | 60         | 35   |
| D       | 10         | 5    |

| Process | Turnaround | Wait |
|---------|------------|------|
| A       |            |      |
| B       |            |      |
| C       |            |      |
| D       |            |      |

<div style="page-break-after:always;"></div>

* Q17: _Draw the schedule when a Round Round (RR) scheduling algorithm is used with a time quantum of 10._

```
  A         B         C         D   A          B    C
|---------|---------|---------|----|---------|----|-------------------|
0         10        20        30   35        45   50                  65
```

<p style="height:8em;"></p>

* Q18: _Compute the turnaround and wait time for each process based on the above schedule._

| Process | Turnaround | Wait |
|---------|------------|------|
| A       | 45         | 25   |
| B       | 50         | 35   |
| C       | 60         | 35   |
| D       | 25         | 20   |

| Process | Turnaround | Wait |
|---------|------------|------|
| A       |            |      |
| B       |            |      |
| C       |            |      |
| D       |            |      |

### Loop optimization

* Q23: _Modify the following code to perform all relevant loop optimizations. You should assume the length of the key is **significantly smaller** than the length of the plain string, such that the entire key will fit in the cache but the entire plain string will not fit in the cache._

In [ ]:
char *vigenere(char *plain, char *key, int keylen) {
    char *cipher = malloc(sizeof(char) * (strlen(plain) + 1));
    cipher[strlen(plain)] = '\0';
    int j = 0;
    while (j < keylen) {
        int i = j;
        while (i < strlen(plain)) {
            cipher[i] = (((plain[i] - 'A') + key[j]) % 26) + 'A';
            i += keylen;
        }
        j++;
    }
    return cipher;
}

Optimizations: loop-invariant code motion to call `strlen` only once; loop interchange to avoid "jumping around" the `plain` array (there is now "jumping around" the key array, but this is tolerable because the key is small enough to fit entirely in the cache)

In [ ]:
char *vigenere(char *plain, char *key, int keylen) {
    int len = strlen(plain);
    char *cipher = malloc(sizeof(char) * (len + 1));
    cipher[len] = '\0';
    int i = 0;
    while (i < len) {
        int j = 0;
        while (j < keylen && i < len) {
            cipher[i] = (((plain[i] - 'A') + key[j]) % 26) + 'A';
            j++;
            i++;
        }
    }
    return cipher;
}

Alternative code with same optimizations

In [ ]:
char *vigenere(char *plain, char *key, int keylen) {
    int len = strlen(plain);
    char *cipher = malloc(sizeof(char) * (len + 1));
    cipher[len] = '\0';
    int i = 0;
    while (i < len) {
        int j = 0;
        while (j < keylen && i+j < len) {
            cipher[i+j] = (((plain[i+j] - 'A') + key[j]) % 26) + 'A';
            j++;
        }
        i += keylen;
    }
    return cipher;
}

<p style="height:20em;"></p>

### Reducing memory accesses

* Q26: _Cross-out unnecessary loads and stores for the following assembly code:_

```
000000000000076c <divide>:                      
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400fe1     ldr    w1, [sp, #8]
    784:    b9400be0     ldr    w0, [sp, #12]
    788:    1ac00c20     sdiv   w0, w1, w0
    78c:    b9001fe0     str    w0, [sp, #28]
    790:    b9401fe0     ldr    w0, [sp, #28]
    794:    910083ff     add    sp, sp, #0x20
    798:    d65f03c0     ret
```

```
000000000000076c <divide>:                      
    76c:    d10083ff     sub    sp, sp, #0x20   
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]        XXXX
    778:    12800000     mov    w0, #0xffffffff     
    77c:    b9001fe0     str    w0, [sp, #28]       XXXX
    780:    b9400fe1     ldr    w1, [sp, #8]        XXXX
    784:    b9400be0     ldr    w0, [sp, #12]
    788:    1ac00c20     sdiv   w0, w1, w0      
    78c:    b9001fe0     str    w0, [sp, #28]       XXXX
    790:    b9401fe0     ldr    w0, [sp, #28]       XXXX
    794:    910083ff     add    sp, sp, #0x20   
    798:    d65f03c0     ret                    
```

* Q27: _Cross-out unnecessary loads and stores for the following assembly code:_

```
000000000000071c <flip>:
    71c:    d10083ff     sub    sp, sp, #0x20
    720:    b9000fe0     str    w0, [sp, #12]
    724:    12800000     mov    w1, #0xffffffff
    728:    b9001fe0     str    w1, [sp, #28]
    72c:    b9400fe0     ldr    w0, [sp, #12]
    730:    7100001f     cmp    w0, #0x0
    734:    54000081     b.eq   740 <flip+0x28>
    738:    b9001fff     str    wzr, [sp, #28]
    73c:    14000002     b      748 <flip+0x2c>
    740:    52800020     mov    w0, #0x1
    744:    b9001fe0     str    w0, [sp, #28]
    748:    b9401fe0     ldr    w0, [sp, #28]
    74c:    910083ff     add    sp, sp, #0x20
    750:    d65f03c0     ret    
```

<div style="page-break-after:always;"></div>

```
000000000000071c <flip>:
    71c:    d10083ff     sub    sp, sp, #0x20
    720:    b9000fe0     str    w0, [sp, #12]   XXXX
    724:    12800000     mov    w1, #0xffffffff
    728:    b9001fe0     str    w1, [sp, #28]
    72c:    b9400fe0     ldr    w0, [sp, #12]   XXXX
    730:    7100001f     cmp    w0, #0x0
    734:    54000081     b.eq   740 <flip+0x28>
    738:    b9001fff     str    wzr, [sp, #28]
    73c:    14000002     b      748 <flip+0x2c>
    740:    52800020     mov    w0, #0x1
    744:    b9001fe0     str    w0, [sp, #28]
    748:    b9401fe0     ldr    w0, [sp, #28]
    74c:    910083ff     add    sp, sp, #0x20
    750:    d65f03c0     ret    
```

### Number representation

* Q37: _Convert `0b10101010` to an unsigned decimal number_

    170

<p style="height:4em;"></p>

* Q38: _Convert `0b10101010` to an signed decimal number_

    -86

<p style="height:4em;"></p>

* Q39: _Convert `43` to 8-bit two's complement_

    0b00101011

<p style="height:4em;"></p>

* Q40: _Convert `-43` to 8-bit two's complement_

    0b11010101

<p style="height:4em;"></p>

* Q41: _Compute `(0b1100 ^ 0b1111) & 0b0101`_

    0b0001

<p style="height:4em;"></p>

* Q42: _Compute `(~0b1100) | 0b0101`_

    0b0111

<p style="height:4em;"></p>

* Q43: _In a system that represents numbers using 8-bit two's complement will the computation `120 + 16` result in overflow, underflow, or neither?_

    overflow

<p style="height:4em;"></p>

* Q44: _In a system that represents numbers using 8-bit two's complement will the computation `-120 + -8` result in overflow, underflow, or neither?_

    neither